<a href="https://colab.research.google.com/github/RCortez25/PhD/blob/main/LLM/4.%20Attention%20mechanism/0_Simplified_attention_mechanism.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The importance that the model gives to a certain word is given by the attention weights.


Self-attention is the key component of modern LLMs. The "self" refers to the ability to compute attention weights by relationg different positions of the same input. Traditional attention mechanisms focus on relationships between elements in different sequences (like input and output).

The goal: Calculate a context vector $z^{(i)}$ for each input vector $x^{(i)}$. This context vector is an enriched version of each input vector which contains information about relationships of tokens.
___

In [ ]:
import torch

# Create the inpur tensors
inputs = torch.tensor(
    [[0.43, 0.15, 0.89], # Your     (x^1)
     [0.55, 0.87, 0.66], # journey  (x^2)
     [0.57, 0.85, 0.64], # starts   (x^3)
     [0.22, 0.58, 0.33], # with     (x^4)
     [0.77, 0.25, 0.10], # one      (x^5)
     [0.05, 0.80, 0.55]] # step     (x^6)
)